### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import scipy.stats as stats
import pylab

In [ ]:
def plot_qq(df,feature):
    plt.figure(figsize = (10,6))
    plt.subplot(1,2,1)
    df[feature].hist()
    plt.subplot(1,2,2)
    stats.probplot(df[feature], dist= 'norm', plot=pylab)
    plt.show()

## Read Data

In [ ]:
#df = pd.read_csv('forbes_billionaires.csv')
df = pd.read_csv("../input/forbes-billionaires-of-2021-20/forbes_billionaires.csv")

categorical_columns = df.dtypes[df.dtypes == 'object'].index
numerical_columns = df.dtypes[df.dtypes != 'object'].index    

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum().plot(kind = 'bar')

In [ ]:
(df.isnull().sum()/df.shape[0]*100).sort_values(ascending = False)

Children and Education columns has more than 40% of the data missing so we will dropt those columns

In [ ]:
df.describe().T

#### Transforming Residence column into City and State column 

In [ ]:
#df["Residence"].value_counts()[df["Residence"].value_counts() > 10]

In [ ]:
df[['City','State','Null']] = df.Residence.apply(lambda x: pd.Series(str(x).split(',')))

In [ ]:
df.head(2)

In [ ]:
df.shape

Here Null column has all the null values so dropping that along with children and education and Residence

In [ ]:
#df[df.Null.notnull()]

Filling State null values with country values

In [ ]:
#df[df.State.isnull()]

In [ ]:
df.loc[df['State'].isnull(),'State'] = df['Country']

Similarly filling null city values with state values; here null values are represented as nan

In [ ]:
df.loc[df['City']== 'nan','City'] = df['State']

#### Dropping columns

In [ ]:
df.drop(['Null','Children','Education','Residence'], axis =1, inplace = True)

Checking rest of the columns

In [ ]:
df.isnull().sum().plot(kind = 'bar')

In [ ]:
df['Status'].value_counts()

Majority of the people are married

In [ ]:
df['Status'].isnull().sum()

In [ ]:
#Adding an extra column where status has null values
#df['Status_Null'] = np.where(df['Status'].isnull(), 1, 0)

In [ ]:
#df['Status_Null'].value_counts()

We will fill the missing values as their own category as Unknown

In [ ]:
df['Status'] = np.where(df['Status'].isnull(), 'Unknown', df['Status'])

In [ ]:
#df.loc[df['Status'].isnull(),'Status'] = 'Unknown'

In [ ]:
df['Status'].value_counts()

In [ ]:
#df.isnull().sum().plot(kind = 'bar')

Now age colunmn

In [ ]:
sns.histplot(x = 'Age', data = df)

In [ ]:
df['Age'].hist()

Looks like normal dist lets check with qq plot

In [ ]:
plot_qq(df,'Age')

It is a normal distribution

add a column which take records age missing values 

In [ ]:
df['Age_Null'] = np.where(df['Age'].isnull(), 1, 0)

replacing age null values with random values from the age distribition

In [ ]:
def na_randomfill(series):
    na_mask = pd.isnull(series)   # boolean mask for null values
    n_null = na_mask.sum()        # number of nulls in the Series
    
    if n_null == 0:
        return series             # if there are no nulls, no need to resample
    
    # Randomly sample the non-null values from our series
    #  only sample this Series as many times as we have nulls 
    fill_values = series[~na_mask].sample(n=n_null, replace=True, random_state=0)

    # This ensures our new values will replace NaNs in the correct locations
    fill_values.index = series.index[na_mask]
    
    return series.fillna(fill_values) 

In [ ]:
df["Age_new"] = na_randomfill(df["Age"])

In [ ]:
# Dropping age columns
df.drop('Age', axis = True, inplace=True)

In [ ]:
#df.isnull().sum().plot(kind = 'bar')

Citizenshipo and self made null values are so low we can directly drop those rows for now and we might work on them later if we want to improve the performance of the mdoel

In [ ]:
df.dropna(axis = 0, how='any', inplace =True)

## EDA

In [ ]:
#### Work in Progress